In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

In [68]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=25)

In [70]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [71]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=2)

Epoch 1/2


d:\Programming\AI-Projects\Learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step - accuracy: 0.5000 - loss: 0.7113
Epoch 2/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5000 - loss: 0.7110


In [72]:
coef, intercept = model.get_weights()

### Custom Neural Network from scratch.

In [81]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [85]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [108]:
class NN():
  def __init__(self) -> None:
    self.w1 = 1
    self.w2 = 1
    self.bias = 0

  def fit(self, x, y, epochs, loss_threshold):
    self.wl, self.w2, self.bias = self.gradient_descent(x['age'], x['affordibility'], y, epochs, loss_threshold)
  
  def predict(self, x_test):
    weighted_sum = self.w1 * x_test['age'] + self.w2 * x_test['affordibility'] + self.bias
    prediction = sigmoid(weighted_sum) * 2
    return prediction
  
  def gradient_descent(self, age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5

    n = len(age)
    for i in range(epochs):
      weighted_sum = w1 * age + w2 * affordability + bias
      y_predicted = sigmoid(weighted_sum)
      loss = log_loss(y_true, y_predicted)

      w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
      w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))
      bias_d = np.mean(y_predicted-y_true)

      w1 = w1 - rate * w1d
      w2 = w2 - rate * w2d
      bias = bias - rate * bias_d

      if i%50==0:
        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
          
      if loss<=loss_thresold:
        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
        break

    return w1, w2, bias

In [109]:
custom_model = NN()
custom_model.fit(X_train_scaled, y_train, 500, .46)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:50, w1:1.503319554173139, w2:1.108384790367645, bias:-1.2319047301235464, loss:0.5675865113475955
Epoch:100, w1:2.200713131760032, w2:1.2941584023238903, bias:-1.6607009122062801, loss:0.5390680417774752
Epoch:150, w1:2.8495727769689085, w2:1.3696895491572745, bias:-1.986105845859897, loss:0.5176462164249294
Epoch:200, w1:3.443016970881803, w2:1.4042218624465033, bias:-2.2571369883752723, loss:0.5005011269691375
Epoch:250, w1:3.982450494649576, w2:1.4239127329321233, bias:-2.494377365971801, loss:0.48654089537617085
Epoch:300, w1:4.472179522095915, w2:1.438787986553552, bias:-2.707387811922373, loss:0.4750814640632793
Epoch:350, w1:4.917245868007634, w2:1.4525660781176122, bias:-2.901176333556766, loss:0.46561475306999006
Epoch:385, w1:5.20486506830687, w2:1.462265634893111, bias:-3.027050433587733, loss:0.4599594305652072


In [110]:
custom_model.predict(X_test_scaled)

2     0.501437
10    0.400483
21    0.118262
11    0.433474
14    0.508988
9     0.555847
dtype: float64

In [113]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64